## [Commands](https://ardupilot.org/dev/docs/copter-commands-in-guided-mode.html)

In [ ]:
from pymavlink import mavutil
import subprocess
import time
import os

Kill related process

In [ ]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Launch Copter (ardupilot)

In [ ]:
sim_vehicle_path = os.path.expanduser("~/ardupilot/Tools/autotest/sim_vehicle.py")
#vehicle_cmd = f"python3 {sim_vehicle_path} -v ArduPlane -L KSFO" 
vehicle_cmd = f"python3 {sim_vehicle_path} -v ArduCopter" 
# Open a new terminal and run the command
subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

<Popen: returncode: None args: ['gnome-terminal', '--', 'bash', '-c', 'pytho...>

Conect to the device

In [ ]:
conn = mavutil.mavlink_connection('udp:127.0.0.1:14550')

conn.wait_heartbeat()
print("Heartbeat received. Connected to the vehicle.")

Heartbeat received. Connected to the vehicle.


In [ ]:
while True:
    try:
        # Get the latest ADSB_VEHICLE message
        adsb_msg = conn.recv_match(type='ADSB_VEHICLE', blocking=True)

        if adsb_msg:
            print(f"ADS-B Target: ICAO Address {adsb_msg.ICAO_address}, Lat: {adsb_msg.lat/1e7}, Lon: {adsb_msg.lon/1e7}")
           # Add your logic to process the ADS-B data here, possibly triggering avoidance maneuvers 
           # based on your configured parameters.
    except KeyboardInterrupt:
        break

print("Exiting...")

# Set required parameters for ADS-B avoidance

In [ ]:
import time
from pymavlink import mavutil

# Connect to vehicle
conn = mavutil.mavlink_connection('udp:127.0.0.1:14550')
conn.wait_heartbeat()
print("✅ Connected to vehicle")

def set_param(name, value, type_):
    conn.mav.param_set_send(
        conn.target_system,
        conn.target_component,
        name.encode('utf-8'),
        value,
        type_
    )
    time.sleep(0.5)

def fetch_param_with_retry(name, retries=5, delay=1):
    for _ in range(retries):
        value = conn.param_fetch_one(name)
        if value is not None:
            return value
        time.sleep(delay)
    return None

def set_and_check_param(name, value, type_):
    set_param(name, value, type_)
    actual = fetch_param_with_retry(name)
    if actual is None:
        print(f"❌ {name} not found after retries")
    elif round(actual, 4) == round(value, 4):
        print(f"✅ {name} = {actual} (OK)")
    else:
        print(f"❌ {name} = {actual} (Expected {value})")

# Set and verify each param
set_and_check_param('AVD_ENABLE', 1, mavutil.mavlink.MAV_PARAM_TYPE_INT32)
set_and_check_param('AVD_F_ACTION', mavutil.mavlink.MAV_COLLISION_ACTION_RTL, mavutil.mavlink.MAV_PARAM_TYPE_INT32)
set_and_check_param('AVD_F_DIST_XY', 1000.0, mavutil.mavlink.MAV_PARAM_TYPE_REAL32)
set_and_check_param('AVD_F_DIST_Z', 100.0, mavutil.mavlink.MAV_PARAM_TYPE_REAL32)
set_and_check_param('AVD_F_TIME', 10.0, mavutil.mavlink.MAV_PARAM_TYPE_REAL32)
set_and_check_param('AVD_F_RCVRY', 1, mavutil.mavlink.MAV_PARAM_TYPE_INT32)



✅ Connected to vehicle
❌ AVD_ENABLE not found after retries
❌ AVD_F_ACTION not found after retries
❌ AVD_F_DIST_XY not found after retries
❌ AVD_F_DIST_Z not found after retries
❌ AVD_F_TIME not found after retries
❌ AVD_F_RCVRY not found after retries


In [5]:
def set_param(name, value, type):
    conn.mav.param_set_send(conn.target_system, conn.target_component,
                            name.encode('utf-8'), value, type)


set_param('AVD_ENABLE', 1, mavutil.mavlink.MAV_PARAM_TYPE_INT32)
set_param('AVD_F_ACTION', mavutil.mavlink.MAV_COLLISION_ACTION_RTL, mavutil.mavlink.MAV_PARAM_TYPE_INT32)
set_param('AVD_F_DIST_XY', 1000.0, mavutil.mavlink.MAV_PARAM_TYPE_REAL32)  # meters
set_param('AVD_F_DIST_Z', 100.0, mavutil.mavlink.MAV_PARAM_TYPE_REAL32)    # meters
set_param('AVD_F_TIME', 10.0, mavutil.mavlink.MAV_PARAM_TYPE_REAL32)       # seconds
set_param('AVD_F_RCVRY', 1, mavutil.mavlink.MAV_PARAM_TYPE_INT32)

print("✅ ADS-B avoidance parameters set")
time.sleep(2)

✅ ADS-B avoidance parameters set


# Get UAV current location

In [6]:

def get_location():
    while True:
        msg = conn.recv_match(type='GLOBAL_POSITION_INT', blocking=True, timeout=5)
        if msg:
            return msg.lat, msg.lon, msg.alt

lat, lon, alt = get_location()
print(f"📍 UAV Location - Lat: {lat}, Lon: {lon}, Alt: {alt}")

📍 UAV Location - Lat: 376193729, Lon: -1223766370, Alt: 5470


# Send a simulated ADS-B aircraft nearby

In [9]:
conn.mav.adsb_vehicle_send(
    42,                            # ICAO address
    lat,                           # same lat
    lon,                           # same lon
    mavutil.mavlink.ADSB_ALTITUDE_TYPE_PRESSURE_QNH,
    alt - 3000,                    # 3 meters below UAV
    18000,                         # heading 180° (centidegrees)
    5000,                          # 50 m/s horizontal speed
    0,                             # vertical speed
    b"threat",                     # callsign
    mavutil.mavlink.ADSB_EMITTER_TYPE_LIGHT,
    1,                             # seconds since last comm
    65535,                         # flags
    1200                           # squawk
)

In [8]:
print("🚨 Simulated fast, close ADS-B aircraft")

# Monitor mode changes
def decode_mode(mode_id):
    mode_map = conn.mode_mapping()
    for name, num in mode_map.items():
        if num == mode_id:
            return name
    return f"Unknown ({mode_id})"

print("⏳ Monitoring for mode change (RTL, AVOID_ADSB, etc.)")
start_mode = None
start_time = time.time()
while time.time() - start_time < 30:
    msg = conn.recv_match(type='HEARTBEAT', blocking=True, timeout=2)
    if msg:
        mode = decode_mode(msg.custom_mode)
        if start_mode is None:
            start_mode = mode
        elif mode != start_mode:
            print(f"🚦 Mode changed: {start_mode} → {mode}")
            break
else:
    print("⚠️ No mode change detected (threat may not be strong enough or parameters not applied)")



🚨 Simulated fast, close ADS-B aircraft
⏳ Monitoring for mode change (RTL, AVOID_ADSB, etc.)
⚠️ No mode change detected (threat may not be strong enough or parameters not applied)


# Get current UAV position

In [19]:
def get_location():
    while True:
        msg = conn.recv_match(type='GLOBAL_POSITION_INT', blocking=True, timeout=5)
        if msg:
            return msg.lat, msg.lon, msg.alt

lat, lon, alt = get_location()
print(f"UAV location: lat={lat}, lon={lon}, alt={alt}")

UAV location: lat=376193729, lon=-1223766372, alt=5410


# Send fake ADS-B vehicle at same location

In [22]:
conn.mav.adsb_vehicle_send(
    12345,                      # ICAO address
    lat,                        # lat (int32, deg * 1e7)
    lon,                        # lon (int32, deg * 1e7)
    mavutil.mavlink.ADSB_ALTITUDE_TYPE_PRESSURE_QNH,  # altitude_type (enum)
    alt,                        # alt (int32, mm)
    0,                          # heading (uint16, centideg)
    0,                          # hor_velocity (cm/s)
    0,                          # ver_velocity (cm/s)
    b"adsbtest",                # callsign (bytes, max 9 chars)
    mavutil.mavlink.ADSB_EMITTER_TYPE_LIGHT,  # emitter_type
    1,                          # tslc (time since last comm)
    65535,                      # flags (bitmask)
    1200                        # squawk (uint16)
)

print("Sent ADS-B threat near UAV")

Sent ADS-B threat near UAV


# Check for a COLLISION message

In [24]:

msg = conn.recv_match(type='COLLISION', blocking=True, timeout=5)
if msg:
    print(f"🚨 COLLISION detected! Action = {msg.action}, Threat Level = {msg.threat_level}")
else:
    print("✅ No collision detected (either safe or avoidance disabled)")

✅ No collision detected (either safe or avoidance disabled)


# Set mode to FBWA

In [17]:
conn.set_mode('AVOID_ADSB')
print("Set to AVOID_ADSB mode")
time.sleep(3)

Set to AVOID_ADSB mode


In [ ]:
def check_noarmed(blocking=False):
    """Check if UAV is already armed"""
    msg = conn.recv_match(type="HEARTBEAT", blocking=blocking)
    if not msg:
        return False  # No message received
    is_armed = msg.base_mode & mavutil.mavlink.MAV_MODE_FLAG_SAFETY_ARMED
    return not is_armed

check_noarmed_step = UAVStep(
    name="Check no armed",   
    check_fn=check_noarmed   # Your function goes here
)
check_noarmed_step.run(blocking=True)
check_noarmed_step.run(blocking=True)
print(check_noarmed_step)

▶️  Starting step: Check no armed
✅ Step done: Check no armed
<UAVStep name='Check no armed', state=DONE, next='None'>


In [7]:

# # ----- Action Class -----
# class UAVAction:
#     def __init__(self, name: str):
#         self.name = name
#         self.current_step = None
#         self.next = None

#     def add_step(self, name, check_fn=None, exec_fn=None):
#         step_number = len(self.steps)
#         step = UAVStep(name, step_number, check_fn, exec_fn)

#         if self.steps:
#             self.steps[-1].next = step
#             step.prev = self.steps[-1]

#         self.steps.append(step)

#         if self.current_step is None:
#             self.current_step = step

#     def link_next(self, next_action):
#         self.next_action = next_action
#         next_action.prev_action = self

#     def reset(self):
#         self.current_step = self.steps[0] if self.steps else None
#         self.state = ACTION_NOT_STARTED

#     def show_steps(self):
#         print(f"\n🧭 Action: {self.name} — {self.state}")
#         for step in self.steps:
#             current = "➡️ " if step == self.current_step else "   "
#             print(f"{current}{step}")

#     def run(self, blocking=True):
#         print(f"\n🚀 Starting action: {self.name}")
#         self.state = ACTION_IN_PROGRESS

#         while self.current_step:
#             step = self.current_step
#             print(f"➡️ Executing: {step}")

#             if not step.check():
#                 print(f"❌ Check failed: {step.name}")
#                 self.state = ACTION_FAILED
#                 return False

#             step.execute(blocking=blocking)
#             print(f"✅ Executed: {step.name}\n")
#             self.current_step = step.next

#         self.state = ACTION_DONE
#         print(f"✅ Action '{self.name}' completed!")
#         return True

#     def run_full(self, blocking=True):
#         current = self
#         while current:
#             current.reset()
#             if not current.run(blocking=blocking):
#                 print("❌ Mission flow interrupted.")
#                 return False
#             current = current.next_action
#         print("🎉 All actions completed successfully!")
#         return True

#     def show_status(self):
#         print(f"📦 Action '{self.name}': {self.state}")

In [8]:
REQUIRED_SENSORS = (
    mavutil.mavlink.MAV_SYS_STATUS_SENSOR_3D_GYRO |
    mavutil.mavlink.MAV_SYS_STATUS_SENSOR_3D_ACCEL |
    mavutil.mavlink.MAV_SYS_STATUS_SENSOR_3D_MAG |
    mavutil.mavlink.MAV_SYS_STATUS_SENSOR_ABSOLUTE_PRESSURE |
    mavutil.mavlink.MAV_SYS_STATUS_SENSOR_GPS
)

EKF_REQUIRED_FLAGS = (
    mavutil.mavlink.EKF_ATTITUDE |
    mavutil.mavlink.EKF_VELOCITY_HORIZ |
    mavutil.mavlink.EKF_POS_HORIZ_ABS |
    mavutil.mavlink.EKF_POS_VERT_ABS
)


class UAV():
    def __init__(self,conn):
        self.conn=conn
        
    def check_heartbeat(blocking=False):
        """Check if UAV is already armed"""
        msg = conn.recv_match(type="HEARTBEAT", blocking=blocking)
        if msg:
            if msg.base_mode & mavutil.mavlink.MAV_MODE_FLAG_SAFETY_ARMED:
                print("❌ UAV is already armed!")
                return False
            print("✅ UAV is disarmed and ready to arm")
            return True
        return False

    def check_ekf_status(blocking=False):
        """Check EKF status for safe arming"""
        msg = conn.recv_match(type="EKF_STATUS_REPORT", blocking=blocking)
        if msg:
            ekf_flags = msg.flags
            # Required EKF flags
            if (ekf_flags & EKF_REQUIRED_FLAGS) == EKF_REQUIRED_FLAGS:
                print("✅ EKF is stable and ready for arming")
                return True
            else:
                print("❌ EKF not ready!")
        return False

    def check_gps_status(self,blocking):
        """Check GPS fix type"""
        msg = self.conn.recv_match(type="GPS_RAW_INT", blocking=blocking)
        if msg:
            gps_fix = msg.fix_type
            if gps_fix >= 3:  # 3D Fix or better
                print(f"✅ GPS Fix OK (Type {gps_fix})")
                return True
            else:
                print(f"❌ GPS Fix is weak (Type {gps_fix})")
        return False

    def check_sys_status(self,blocking):
        """Check system status (failsafe, battery, sensors)"""
        msg = self.conn.recv_match(type="SYS_STATUS", blocking=blocking)
        if msg:
            # Battery check
            if msg.battery_remaining > 20:  # More than 20% battery
                print(f"✅ Battery OK ({msg.battery_remaining}% remaining)")
                return True
            else:
                print(f"❌ Battery too low ({msg.battery_remaining}% remaining)")
            
            # Sensor check
            if msg and (msg.onboard_control_sensors_health & REQUIRED_SENSORS) == REQUIRED_SENSORS:
                print("✅ All required sensors are healthy! Ready for GPS-based autonomous flight")
                return True
            else:
                print("❌ Missing or unhealthy sensors! Not safe for autonomous flight")
        return False




## Prearmed Comands

In [9]:
# Run all pre-arm checks
uav=UAV(conn)

while True:
    if uav.check_heartbeat():
        break
while True:
    if uav.check_sys_status():
        break
while True:
    if uav.check_ekf_status():
        break
while True:
    if uav.check_gps_status():
        break

print("\n✅✅✅ UAV is READY TO ARM!\n")


✅ UAV is disarmed and ready to arm


TypeError: check_sys_status() missing 1 required positional argument: 'blocking'

## Movement Commands

These commands can be used to control the vehicle’s position, velocity or attitude while in [Guided Mode](https://ardupilot.org/copter/docs/ac2_guidedmode.html#ac2-guidedmode)

In [ ]:
mavutil.mavlink.EKF_POS_HORIZ_ABS

16